In [1]:
from pymongo import MongoClient
from gridfs import GridFS

import pandas as pd

import tqdm
import requests
import urllib.request as request
from bs4 import BeautifulSoup
from progressbar import ProgressBar
import json

In [2]:
# 建立mongodb连接
# mongoClient = MongoClient('mongodb://localhost:27017/')  
mongoClient = MongoClient('mongodb://172.10.10.47:27019/') 
dtmr_dev = mongoClient.get_database('dtmr_dev')
dtmr_dev.authenticate('dtmr_dev', 'dtmr_dev')
dev = dtmr_dev.get_collection('TargetInfo')

In [3]:
UPIDs = pd.read_excel("target_ID.xlsx").sort_values(by="No.").set_index("No.").Human_UPID.values

In [4]:
for UPID in tqdm.tqdm(UPIDs):
    html_url = "https://www.uniprot.org/uniprot/" + UPID
    response = request.urlopen(html_url)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    
    alternateNames = []
    for name_div in soup.find_all("div", property="alternateName"):
        alternateNames.append(name_div.text.split("Short name:")[0])
    
    query = { "UPID": UPID }
    newvalues = { "$set": { "alternateName": alternateNames } }

    dev.update_one(query, newvalues)

100%|████████████████████████████████████████████████████████████████████████████████| 536/536 [34:10<00:00,  3.83s/it]
